In [1]:
import requests
import time
#from time import strftime
import datetime
import numpy as np
import re
import xmltodict
import pprint# as pp
#import process_command
#print(dir(process_command))

In [2]:
def openWebsite(adress):
    r = requests.get(adress)
    return r.text

In [3]:
def prettyPrint(text):
    pp = pprint.PrettyPrinter(indent=1)
    pp.pprint(text)

In [4]:
def process_command_vbb(command, param):  
    def furtherParamString(param):#,listToDelete):
        ret=""
        for key in param.keys():
            ret+= "&"+str(key)+"="+str(param[key])
        return ret
    beforeId=command
    behindId=furtherParamString(param)
    return {"beforeId":beforeId, "behindId":behindId}

In [5]:
def createAdress_vbb(command, param):
    baseurl="http://demo.hafas.de/openapi/vbb-proxy/"
    commands = process_command_vbb(command, param)
    accessId ="felix-fauer-8b71-1705950c2589"
    adress = baseurl + commands['beforeId'] + "?accessId=" + accessId + commands['behindId']
    print(adress)
    return adress

In [15]:
def responseToDict(response):
    print(response)
    return xmltodict.parse(response)

In [7]:
def getData(command, param):
    response_xml = openWebsite(createAdress_vbb(command, param))
    return responseToDict(response_xml)    

In [8]:
def nameToExt(name):
    response = getData('location.name', {"input":name,"maxNo":1})
    return response['LocationList']['StopLocation']['@extId']

In [9]:
def short_departureAt(name, maxNo=3):
    ext=nameToExt(name)
    command, param ='departureBoard', {'extId':ext,'maxJourneys':maxNo}
    response = getData(command, param)
    return response

In [10]:
def nice_nextXDeparturesAt(name, maxNo=3):
    response = short_departureAt(name, maxNo)
    print("Next departures at "+str(response['DepartureBoard']['Departure'][0]['@stop'])+":\n")
    print("   Line        Destination                     Time                  (intern data)")
    print("   ----        -----------                     ----")
    for dep in response['DepartureBoard']['Departure']:
        if "@rtTime" in dep.keys():
            if dep['@time'][0:5]!= dep['@rtTime'][0:5]:
                print("   "+dep['@name'].lstrip().ljust(8)+"    "+dep['@direction'].ljust(30)[0:30]+"  "+
                      dep['@time'][0:5]+" ("+dep['@rtTime'][0:5]+")"+"".ljust(9)+dep['JourneyDetailRef']['@ref'])
            else: 
                print("   "+dep['@name'].lstrip().ljust(8)+"    "+dep['@direction'].ljust(30)[0:30]+"  "+
                      dep['@rtTime'][0:5]+"       "+"".ljust(10)+dep['JourneyDetailRef']['@ref'])
        else:
            print("   "+dep['@name'].lstrip().ljust(8)+"    "+dep['@direction'].ljust(30)[0:30]+"  "+
                  dep['@time'][0:5]+" (?)    "+"".ljust(9)+dep['JourneyDetailRef']['@ref'])


In [11]:
#nice_nextXDeparturesAt("Tauernallee/",5)
nice_nextXDeparturesAt("Messe Nord",20)

http://demo.hafas.de/openapi/vbb-proxy/location.name?accessId=felix-fauer-8b71-1705950c2589&input=Messe Nord&maxNo=1
http://demo.hafas.de/openapi/vbb-proxy/departureBoard?accessId=felix-fauer-8b71-1705950c2589&extId=900024106&maxJourneys=20
Next departures at S Messe Nord/ICC (Berlin):

   Line        Destination                     Time                  (intern data)
   ----        -----------                     ----
   M49         S+U Zoologischer Garten         11:34 (?)             1|24918|3|86|20022022
   M49         Staaken, Reimerweg              11:40 (?)             1|24963|4|86|20022022
   X34         S+U Zoologischer Garten         11:43 (?)             1|27952|3|86|20022022
   M49         S+U Zoologischer Garten         11:44 (?)             1|24845|4|86|20022022
   X34         Kladow, Hottengrund             11:46 (?)             1|28070|0|86|20022022
   M49         Heerstr./Nennhauser Damm        11:50 (?)             1|25037|4|86|20022022
   139         Hakenfelde, Werd

In [16]:
# does not work yet. responste from site is not in the expected format. click link.

def get_stations_of_line():
    ext=900024106
    maxNo=5
    command, param ='trainSearch', {'match':'S1','date':'2022-02-20', 'extId':ext,'maxJourneys':maxNo}
    response = getData(command, param)
    
    #return(response)
get_stations_of_line()

http://demo.hafas.de/openapi/vbb-proxy/trainSearch?accessId=felix-fauer-8b71-1705950c2589&match=S1&date=2022-02-20&extId=900024106&maxJourneys=5
<html><head><meta charset="UTF-8"></head><body><ul><li><a href='https://demo.hafas.de/openapi/vbb-proxy/arrivalBoard?wadl' target='_blank'>Arrival board</a></li><li><a href='https://demo.hafas.de/openapi/vbb-proxy/departureBoard?wadl' target='_blank'>Departure board</a></li><li><a href='https://demo.hafas.de/openapi/vbb-proxy/gisroute?wadl' target='_blank'>Gis Route</a></li><li><a href='https://demo.hafas.de/openapi/vbb-proxy/himsearch?wadl' target='_blank'>HIM Search</a></li><li><a href='https://demo.hafas.de/openapi/vbb-proxy/journeyDetail?wadl' target='_blank'>Journey detail</a></li><li><a href='https://demo.hafas.de/openapi/vbb-proxy/location.nearbystops?wadl' target='_blank'>Location search by coordinate</a></li><li><a href='https://demo.hafas.de/openapi/vbb-proxy/location.name?wadl' target='_blank'>Location search by name</a></li><li><a 

ExpatError: mismatched tag: line 1, column 36

In [13]:
#<baseurl>/trainSearch?accessId=abc&match=S4&date=2016-11-14